In [ ]:
!pip install -q "protobuf==3.20.*"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, bu

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional, Input
import json
import os
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

2025-11-30 14:46:19.364594: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764513979.511954      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764513979.557742      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
diacritic2id_path = "/kaggle/input/arabicia-2/diacritic2id.json"
idx2char_path = "/kaggle/input/arabicia-2/idx2char.json"
char2idx_path = "/kaggle/input/arabicia-2/char2idx.json"

y_train_path = "/kaggle/input/arabicia-2/cleaned_tashkeel.txt"
y_val_path = "/kaggle/input/arabicia-2/cleaned_tashkeel_val.txt"

padded_train = "/kaggle/input/padded/padded.pkl"
padded_val = "/kaggle/input/padded/padded_val.pkl"

In [ ]:
with open(diacritic2id_path, 'r', encoding='utf-8') as f:
    diacritic2id = json.load(f)

id2label = {v: k for k, v in diacritic2id.items()}

with open(char2idx_path, 'r', encoding='utf-8') as f:
    char2idx = json.load(f)

with open(idx2char_path, 'r', encoding='utf-8') as f:
    idx2char = json.load(f)

idx2char['1'] = '?'
value = char2idx.pop('\uFFFD')
char2idx['?'] = value

In [ ]:
def recover(sentence, tashkeel, testing=False):
    result = ""
    i = 0
    while i < len(sentence):
        result += sentence[i] + tashkeel[i]
        i += 1

    return result

In [ ]:
def load_data_pickle(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)
    X = []
    y = []

    for text_seq, label_seq in zip(X_raw, y_raw):
        x_ids = [c for c in text_seq]
        y_ids = [t for t in label_seq]

        X.append(x_ids)
        y.append(y_ids)

    return np.array(X), np.array(y)

In [ ]:
def remove_pads(sentence, tashkeel):
    chars = []
    tashkeelat = []
    for i in range(len(sentence)):
        if sentence[i] != '<PAD>':
            chars.append(sentence[i])
            tashkeelat.append(tashkeel[i])
    sentence_text = "".join(chars)
    text = sentence_text.replace('\uFFFD', '?')
    return text, tashkeelat

In [ ]:
sentences, tashkeel_sequences = load_data_pickle(padded_train)

In [ ]:
new_sentences = []
new_tashkeel = []
for i in range(len(sentences)):
    text, tashkeel = remove_pads(sentences[i], tashkeel_sequences[i])
    new_sentences.append(text)
    new_tashkeel.append(tashkeel)

In [ ]:
sentence_lengths = [len(sentence) for sentence in new_sentences]

In [ ]:
files_config = []
for i in range(0, 64001, 16000):
    files_config.append((f'/kaggle/input/arabic-diacrirization-features-{i + 16000}/zizo_part_{i + 16000}.h5', sentence_lengths[i:i + 16000]))
files_config.append((f'/kaggle/input/arabic-diacrirization-features-80254/zizo_part_80254.h5', sentence_lengths[80000:80254]))

In [ ]:
MAX_SEQ_LEN = 807
CHAR_EMBED_DIM = 256
BATCH_SIZE = 32
EPOCHS = 10
VOCAB_SIZE = len(char2idx)
NUM_CLASSES = len(diacritic2id)
DIRTY = False
DIACRITICS_PATTERN = re.compile(r'[\u064B-\u0652]')
INPUT_DIM = 1024 # bert + mine
HIDDEN_DIM = 512
total_sentences = sum([len(lengths) for _, lengths in files_config])
STEPS_PER_EPOCH = total_sentences // BATCH_SIZE
PADDING_INPUT = -99999.0

In [ ]:
class SparseCategoricalFocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, ignore_id=15, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma
        self.ignore_id = ignore_id

    def call(self, y_true, y_pred):
        y_true = tf.squeeze(y_true, axis=-1)
        y_true = tf.cast(y_true, tf.int32)

        mask = tf.cast(tf.not_equal(y_true, self.ignore_id), tf.float32)

        y_true_safe = tf.where(y_true == self.ignore_id, 0, y_true)

        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_safe, y_pred, from_logits=False)

        p_t = tf.math.exp(-loss)
        focal_loss = tf.math.pow(1.0 - p_t, self.gamma) * loss

        masked_loss = focal_loss * mask

        return tf.reduce_sum(masked_loss) / (tf.reduce_sum(mask) + 1e-6)

In [ ]:
import h5py
def multi_file_generator():
    for h5_path, lengths in files_config:

        with h5py.File(h5_path, 'r') as f:
            features_dset = f['features']
            labels_dset = f['tashkeel']

            current_idx = 0

            for seq_len in lengths:
                end_idx = current_idx + seq_len

                feat = features_dset[current_idx : end_idx]
                lbl = labels_dset[current_idx : end_idx]

                if feat.shape[0] > 0:
                    lbl = lbl.reshape(-1, 1)
                    yield feat, lbl

                current_idx = end_idx

dataset = tf.data.Dataset.from_generator(
    multi_file_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, INPUT_DIM), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.int32)
    )
)

In [ ]:
dataset = dataset.padded_batch(
    BATCH_SIZE,
    padding_values=(PADDING_INPUT, 15)
)
dataset = dataset.repeat()

dataset = dataset.prefetch(tf.data.AUTOTUNE)

def build_model():
    inputs = tf.keras.Input(shape=(None, INPUT_DIM))

    masked = tf.keras.layers.Masking(mask_value=PADDING_INPUT)(inputs)

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_DIM, return_sequences=True))(masked)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_DIM, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    # x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_DIM, return_sequences=True))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_DIM, return_sequences=True))(x)

    x = tf.keras.layers.Dense(HIDDEN_DIM, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def masked_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true = tf.cast(y_true, tf.int32)

    mask = tf.cast(tf.not_equal(y_true, 15), tf.float32)

    y_pred_class = tf.cast(tf.argmax(y_pred, axis=-1), tf.int32)

    correct = tf.cast(tf.equal(y_true, y_pred_class), tf.float32)

    masked_correct = correct * mask

    return tf.reduce_sum(masked_correct) / (tf.reduce_sum(mask) + 1e-6)

In [ ]:
model = build_model()

model.compile(
    optimizer='adam',
    loss=SparseCategoricalFocalLoss(gamma=2.0), # ,ignore_id=14),
    metrics=[masked_accuracy]
)

model.fit(dataset,
          steps_per_epoch=STEPS_PER_EPOCH,
          epochs=10,
          verbose=True)

Epoch 1/15


I0000 00:00:1764514749.233684     138 cuda_dnn.cc:529] Loaded cuDNN version 90300


2007/2007 ━━━━━━━━━━━━━━━━━━━━ 586s 288ms/step - loss: 0.2888 - masked_accuracy: 0.8380
Epoch 2/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - loss: 0.0593 - masked_accuracy: 0.4741    
Epoch 3/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2007/2007 ━━━━━━━━━━━━━━━━━━━━ 578s 288ms/step - loss: 0.0410 - masked_accuracy: 0.9652
Epoch 4/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 0s 5us/step - loss: 0.0391 - masked_accuracy: 0.4827    
Epoch 5/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 579s 288ms/step - loss: 0.0278 - masked_accuracy: 0.9747
Epoch 6/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 0s 5us/step - loss: 0.0276 - masked_accuracy: 0.4880    
Epoch 7/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 580s 289ms/step - loss: 0.0210 - masked_accuracy: 0.9794
Epoch 8/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 0s 5us/step - loss: 0.0224 - masked_accuracy: 0.4901    
Epoch 9/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 580s 289ms/step - loss: 0.0167 - masked_accuracy: 0.9825
Epoch 10/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 0s 5us/step - loss: 0.0158 - masked_accuracy: 0.4913    
Epoch 11/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 580s 289ms/step - loss: 0.0138 - masked_accuracy: 0.9848
Epoch 12/15
2007/2007 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - loss: 0.0159 - masked_accuracy: 0.4915    
Epoch 13/15
2007/2

# Testing

In [ ]:
def another_smart_slicer_generator(lengths, offset=0):
    with h5py.File('/kaggle/input/arabic-diacrirization-features-val-3907/zizo_part_3907.h5', 'r') as f:
        features_dset = f['features']
        labels_dset = f['tashkeel']

        current_idx = offset

        for seq_len in lengths:
            end_idx = current_idx + seq_len

            feat = features_dset[current_idx : end_idx]
            lbl = labels_dset[current_idx : end_idx]

            lbl = lbl.reshape(-1, 1)

            yield feat, lbl

            current_idx = end_idx

In [ ]:
sentences, tashkeel_sequences = load_data_pickle(padded_val)
new_sentences = []
new_tashkeel = []
for i in range(len(sentences)):
    text, tashkeel = remove_pads(sentences[i], tashkeel_sequences[i])
    new_sentences.append(text)
    new_tashkeel.append(tashkeel)
sentence_lengths = [len(sentence) for sentence in new_sentences]

In [ ]:
test_dataset = tf.data.Dataset.from_generator(
    lambda: another_smart_slicer_generator(all_sentence_lengths, offset=0),
    output_signature=(
        tf.TensorSpec(shape=(None, INPUT_DIM), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.int32)
    )
).padded_batch(BATCH_SIZE, padding_values=(PADDING_INPUT, 15))

In [ ]:
total_correct = 0
total_chars = 0
sentences_processed = 0

for batch_x, batch_y in test_dataset:

    batch_probs = model.predict_on_batch(batch_x)

    batch_pred_ids = np.argmax(batch_probs, axis=-1)

    batch_true_ids = batch_y.numpy().squeeze(axis=-1)

    batch_size = batch_pred_ids.shape[0]

    for k in range(batch_size):

        valid_indices = np.where(batch_true_ids[k] != 15)[0]

        if len(valid_indices) == 0:
            continue

        real_len = valid_indices[-1] + 1

        p_seq = batch_pred_ids[k][:real_len]
        t_seq = batch_true_ids[k][:real_len]

        matches = np.sum(p_seq == t_seq)

        total_correct += matches
        total_chars += real_len
        sentences_processed += 1

if total_chars > 0:
    print(f"Final Accuracy: {total_correct / total_chars:.2%}")

# Save the model

In [ ]:
# Standard Keras format (Recommended)
model.save('model_with_features_v1.keras')

In [ ]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

def run_test_inference(test_sentences, bert_model, bert_tokenizer, classifier_model, batch_size=32):

    all_features = []
    all_lengths = []

    print("Extracting features...")
    for i, sent in tqdm(enumerate(test_sentences), total=len(test_sentences)):
        feat, _ = zizo_features(sent, i, bert_model, bert_tokenizer)

        all_features.append(feat)
        all_lengths.append(len(feat))

    def generator():
        for feat in all_features:
            yield feat

    feature_dim = all_features[0].shape[1]

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=tf.TensorSpec(shape=(None, feature_dim), dtype=tf.float32)
    )

    dataset = dataset.padded_batch(batch_size, padding_values=0.0)

    print("Running inference...")
    predictions = classifier_model.predict(dataset, verbose=1)

    predicted_ids = np.argmax(predictions, axis=-1)

    final_diacritized_sentences = []

    print("Reconstructing text...")
    for i, length in enumerate(all_lengths):
        sentence_pred_ids = predicted_ids[i][:length]

        original_text = test_sentences[i]

        res = ""
        for char_idx, char in enumerate(original_text):
            if char_idx < len(sentence_pred_ids):
                d_id = sentence_pred_ids[char_idx]
                diacritic = id2diacritic[d_id] if d_id in id2diacritic else ""
                res += char + diacritic
            else:
                res += char

        final_diacritized_sentences.append(res)

    return final_diacritized_sentences